In [1]:
import os
import sys
import numpy as np
import cv2
import time
from collections import Counter
#Import mrcnn module
import mrcnn.config as conf
import mrcnn.model as model
import visualize_1 as visual
import skimage.io


ROOT_DIR=os.path.abspath("/home/noor/projects/car parking/")

#Directory to save trained model
MODEL_DIR=os.path.join(ROOT_DIR,"model")

#Coco weights file path
WEIGHT_DIR=os.path.join(ROOT_DIR,"mask_rcnn_coco.h5")


Using TensorFlow backend.


In [2]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [3]:
def colors(class_names):
    np.random.seed(12)
    rand_colors=[tuple(np.random.rand(3)*255) for _ in range(len(class_names))]
    return rand_colors
colour=colors(class_names)
dict_color={ID:color for (ID,color) in enumerate(colour) }

   

In [4]:
def apply_mask(image,mask,color,alpha):
    for i in range(3):
        image[:,:,i]=np.where(mask==1,image[:,:,i]*(1-alpha)+alpha*color[i],image[:,:,i])
    return image

In [7]:
def displayinstances(image,masks,class_ids,class_names,scores,boxes):
    N=boxes.shape[0]
    if not N:
        pass
    else:
        assert  N==masks.shape[-1]==class_ids.shape[0]
        
    
    for i in range(N):
         if not np.any(boxes[i]) or label!="car" or label!="bicycle" or label!="motorcycle" or label!="bus" or label!="truck":
            # Skip this instance. Has no bbox. Likely lost in image cropping or label is not required vehicle
             continue
         class_id = class_ids[i]       
         label = class_names[class_id]
         y1, x1, y2, x2 = boxes[i]
         score = scores[i] if scores is not None else None
         color = dict_color[class_id]
         caption = "{} {:.3f}".format(label, score) if score else label
         mask = masks[:, :, i]
         image = apply_mask(image, mask, color,0.5)
         image = cv2.rectangle(image,(x1,y1),(x2,y2),color,2)
         image = cv2.putText(image,caption,(x1,y1),cv2.FONT_HERSHEY_PLAIN,0.5,color,2)
    return image

In [8]:
#Inherit from base class Config
class parkconfig(conf.Config):
    #batch_size=GPU_COUNT*IMAGES_PER_GPU
    #GPU_COUNT=1
      NAME="PARK"
      IMAGES_PER_GPU= 1
      NUM_CLASSES=80+1
config=parkconfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [9]:
#create inference(testing) model
Model=model.MaskRCNN(mode="inference",config=config,model_dir=MODEL_DIR)

#load pretrained weights for the model
Model.load_weights(filepath=WEIGHT_DIR,by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [ ]:
cap = cv2.VideoCapture("car_highway.mp4")
while(cap.isOpened()):#returns true if capture is initialized already
    ret,frame=cap.read()#read frame
    if ret:
        output=Model.detect(images=[frame],verbose=0)#return list of dictionaries(one dict per image)
        out=output[0]#first element of the list
        image=displayinstances(frame,out["masks"],out["class_ids"],class_names,out["scores"],out["rois"])
        
        RGB_img= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.imshow("image",RGB_img)
    if cv2.waitKey(100) & 0xFF==ord("q"):#waits for 1 millisecond or press q to exit
        break     
cap.release()#close the video file or capturing device(s)
cv2.destroyAllWindows()

